In [ ]:
# This line solves some minor problems when you do not have propery set the PYTHONPATH
exec(compile(open("fix_paths.py", "rb").read(), "fix_paths.py", 'exec'))

## Load data example

In [ ]:
from common import dataset_loaders
glabels = dataset_loaders.groundlabels_dataframe()

map_category = {'A':'Moto', 'B':'Car', 'C':'Car', 'D':'Car', 'E':'Car', 'F':'Car', 'G':'Car', 'H':'Van', 'I':'Bus'}
glabels2 = dataset_loaders.map_labels(glabels, map_category)

# Plot data example

In [ ]:
from common import plot_utils
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 15)

In [ ]:
#casename = dataset_loaders.get_casenames()[0]
#img = dataset_loaders.load_image(casename)
#labels = glabels[glabels['image'] == casename]
#plot_utils.draw(img, labels)

In [ ]:
#casenames = dataset_loaders.get_casenames()[:10]
#imgs = [dataset_loaders.load_image(casename) for casename in casenames]
#labs = [glabels[glabels['image'] == casename] for casename in casenames]
#plot_utils.multidraw([5,4], imgs, labs)

# Example to use loaders

In [ ]:
casename = dataset_loaders.get_casenames()[0]
img = dataset_loaders.load_image(casename)
labels = glabels[glabels['image'] == casename]

# Create generators


In [ ]:
import settings
import os
import pandas as pd
from common import dataset_loaders
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from patch_generators.pos_and_negative_fix_size import LabelEncoding, data_generator

experiment_name = 'vehicle_empty_resnet_v0'

# We create the data structure we need
experiment_folder_name = 'patches_single_size'
os.system('mkdir -p %s/%s' % (settings.DATAMODEL_PATH, experiment_folder_name))
os.system('mkdir -p %s/%s/models' % (settings.DATAMODEL_PATH, experiment_folder_name))
os.system('mkdir -p %s/%s/logs' % (settings.DATAMODEL_PATH, experiment_folder_name))

OUTPUT_MODEL = '%s/%s/models/vehicle_empty_discriminator.hdf5' % (settings.DATAMODEL_PATH, experiment_folder_name)
LOGS_PATH    = '%s/%s/logs/%s' % (settings.DATAMODEL_PATH, experiment_folder_name, experiment_name)



image_size_nn = 48
num_valid_cases = 60
patch_size = 110
batch_size = 25

restart_valid_train = True


def load_labels(restart_valid_train):
    # Generate consistent train and validation sets
    ## Only do this if the train / validation has not been generated yet
    original_labels = dataset_loaders.groundlabels_dataframe()
    if restart_valid_train == True:
        os.system('rm -f %s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))
        os.system('rm -f %s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))

    if not os.path.exists('%s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name)) and not os.path.exists('%s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name)):
        cases = original_labels['image'].unique()
        np.random.shuffle(cases)
        train_data = original_labels[original_labels.image.isin(cases[num_valid_cases:])]
        valid_data = original_labels[~original_labels.image.isin(cases[num_valid_cases:])]

        train_data.to_csv('%s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name), index = False)
        valid_data.to_csv('%s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name), index = False)
    train_data = pd.read_csv('%s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))
    valid_data = pd.read_csv('%s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))
    return original_labels, train_data, valid_data

original_labels, train_data, valid_data = load_labels(restart_valid_train)

# Initialize these cases
## We add background as label to the rest of existing labels
existing_labels = np.concatenate([original_labels['class'].unique(), ['background']])
labelencoder = LabelEncoding(existing_labels)
data_augmentation = ImageDataGenerator(vertical_flip=True, horizontal_flip = True, zoom_range = 0.02, rotation_range=180)

train_generator = data_generator(data_augmentation, labelencoder, train_data, batch_size, patch_size, image_size_nn)
valid_generator = data_generator(data_augmentation, labelencoder, valid_data, batch_size, patch_size, image_size_nn)

#from common import plot_utils
#%matplotlib inline
#%pylab inline
#pylab.rcParams['figure.figsize'] = (15, 15)
#for a,b in train_generator:
#    break
#plot_utils.multidraw([10,5], a)
#[labelencoder.get_class_id(rw) for rw in b][:4]

In [ ]:
for a,b in valid_generator:
    break

In [ ]:
a.shape

# TRAIN PROCESS

In [2]:
# This line solves some minor problems when you do not have propery set the PYTHONPATH
exec(compile(open("fix_paths.py", "rb").read(), "fix_paths.py", 'exec'))

import settings
import os
import pandas as pd
from common import dataset_loaders
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from patch_generators.pos_and_negative_fix_size import LabelEncoding, data_generator
import time

experiment_name = 'vehicle_empty_resnet_v0'

# We create the data structure we need
experiment_folder_name = 'patches_single_size'
os.system('mkdir -p %s/%s' % (settings.DATAMODEL_PATH, experiment_folder_name))
os.system('mkdir -p %s/%s/models' % (settings.DATAMODEL_PATH, experiment_folder_name))
os.system('mkdir -p %s/%s/logs' % (settings.DATAMODEL_PATH, experiment_folder_name))

OUTPUT_MODEL = '%s/%s/models/vehicle_empty_discriminator.hdf5' % (settings.DATAMODEL_PATH, experiment_folder_name)
LOGS_PATH    = '%s/%s/logs/%s' % (settings.DATAMODEL_PATH, experiment_folder_name, experiment_name)



image_size_nn = 48
num_valid_cases = 60
patch_size = 110
batch_size = 25
big_batch_size, valid_batch_size = 100, 500

restart_valid_train = True


def load_labels(restart_valid_train):
    # Generate consistent train and validation sets
    ## Only do this if the train / validation has not been generated yet
    original_labels = dataset_loaders.groundlabels_dataframe()
    if restart_valid_train == True:
        os.system('rm -f %s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))
        os.system('rm -f %s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))

    if not os.path.exists('%s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name)) and not os.path.exists('%s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name)):
        cases = original_labels['image'].unique()
        np.random.shuffle(cases)
        train_data = original_labels[original_labels.image.isin(cases[num_valid_cases:])]
        valid_data = original_labels[~original_labels.image.isin(cases[num_valid_cases:])]

        train_data.to_csv('%s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name), index = False)
        valid_data.to_csv('%s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name), index = False)
    train_data = pd.read_csv('%s/%s/train_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))
    valid_data = pd.read_csv('%s/%s/valid_df.csv' % (settings.DATAMODEL_PATH, experiment_folder_name))
    return original_labels, train_data, valid_data

original_labels, train_data, valid_data = load_labels(restart_valid_train)

# Initialize these cases
## We add background as label to the rest of existing labels
existing_labels = np.concatenate([original_labels['class'].unique(), ['background']])
labelencoder = LabelEncoding(existing_labels)
data_augmentation = ImageDataGenerator(vertical_flip=True, horizontal_flip = True, zoom_range = 0.02, rotation_range=180)

train_generator = data_generator(data_augmentation, labelencoder, train_data, batch_size=big_batch_size, min_buffer_before_start = 5000, patch_size=patch_size, image_size_nn=image_size_nn)
valid_generator = data_generator(None, labelencoder, valid_data, batch_size=valid_batch_size,  min_buffer_before_start = 1000, patch_size=patch_size, image_size_nn=image_size_nn)


Using Theano backend.
ERROR (theano.sandbox.cuda): Failed to compile cuda_ndarray.cu: libcublas.so.8.0: cannot open shared object file: No such file or directory


In [3]:
for x,y in valid_generator:
    print(x.shape)

(500, 3, 48, 48)
(500, 3, 48, 48)
(500, 3, 48, 48)


FileNotFoundError: [Errno 2] No such file or directory: '/home/jose/tech/ml_projects/datasets/cars_contest/test/TQ3477_4_7.jpg'

In [ ]:
model